In [73]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [39]:
homes_B = pd.read_pickle("B_homes.pickle")
homes_G = pd.read_pickle("G_homes.pickle")

names_edges = ["from_id", "to_id"]

edges_B = pd.read_table("Brightkite_edges.txt", names=names_edges).dropna()
edges_G = pd.read_table("Gowalla_edges.txt", names=names_edges).dropna()

# since the edges are bidrectional (and we only need one direction) we remove half the 
# values
edges_B[["from_id", "to_id"]] = np.sort(edges_B[["from_id", "to_id"]].values, axis=1)
edges_B = edges_B.drop_duplicates()
edges_G[["from_id", "to_id"]] = np.sort(edges_G[["from_id", "to_id"]].values, axis=1)
edges_G = edges_G.drop_duplicates()

edges_B

,from_id,to_id
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
428142,58219,58224
428144,58220,58225
428145,58220,58226
428151,58225,58226


# Go through the edges dataframe and replace user id's with corresponding country codes

In [82]:
from_country = []
to_country = []
for row in tqdm(edges_B.iterrows()):
    from_id = row[1]["from_id"]
    to_id = row[1]["to_id"]
    if from_id in homes_B.index and to_id in homes_B.index: # TODO leave out if no home addresses
        from_country.append(homes_B.loc[from_id]["country"][0])
        to_country.append(homes_B.loc[to_id]["country"][0])

214078it [01:48, 1969.02it/s]


### Build Dataframe and Count Occurrences

In [101]:
edges_countries = pd.DataFrame(list(zip(from_country, to_country)), 
                                   columns =["from_country", "to_country"]) 
# count 
edges_countries = edges_countries.groupby(["from_country", "to_country"]).size().reset_index(name='Count')
# remove country to oneself
edges_countries = edges_countries[edges_countries['from_country'] != edges_countries['to_country']]
edges_countries

,from_country,to_country,Count
1,AE,AT,1
2,AE,AU,3
3,AE,BH,1
4,AE,BR,1
5,AE,CA,1
...,...,...,...
1798,ZA,RU,1
1799,ZA,SD,1
1800,ZA,TH,1
1801,ZA,US,12


In [102]:
edges_countries[edges_countries["from_country"] == "US"].sort_values(["Count"], ascending=False)

,from_country,to_country,Count
1646,US,GB,2851
1625,US,CA,2096
1612,US,AU,1423
1669,US,JP,1244
1635,US,DE,676
...,...,...,...
1721,US,SN,1
1666,US,JE,1
1692,US,MZ,1
1725,US,TD,1


In [111]:
country = "US"
edges_countries[(edges_countries["from_country"]=="US") | (edges_countries["to_country"]=="US")]

,from_country,to_country,Count
24,AE,US,30
41,AR,US,19
81,AT,US,75
139,AU,US,635
145,BA,US,1
...,...,...,...
1749,UY,US,26
1765,VA,US,11
1771,VI,US,1
1775,VN,US,3


In [110]:
edges_countries["from_country"]=="US" & edges_countries["from_country"]=="US"

TypeError: Cannot perform 'rand_' with a dtyped [object] array and scalar of type [bool]